## Preprocessing

In [16]:
#! git clone https://github.com/Leena-680/deep-learning-challenge.git

Cloning into 'deep-learning-challenge'...


In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import dask.dataframe as dd
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.callbacks import ModelCheckpoint

#  Import and read the charity_data.csv.
application_df = pd.read_csv("Resources/charity_data.csv")
#application_df.head()

#application_df = pd.read_csv('/content/deep-learning-challenge/Resources/charity_data.csv')

# review the DataFrame
application_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34299 entries, 0 to 34298
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   EIN                     34299 non-null  int64 
 1   NAME                    34299 non-null  object
 2   APPLICATION_TYPE        34299 non-null  object
 3   AFFILIATION             34299 non-null  object
 4   CLASSIFICATION          34299 non-null  object
 5   USE_CASE                34299 non-null  object
 6   ORGANIZATION            34299 non-null  object
 7   STATUS                  34299 non-null  int64 
 8   INCOME_AMT              34299 non-null  object
 9   SPECIAL_CONSIDERATIONS  34299 non-null  object
 10  ASK_AMT                 34299 non-null  int64 
 11  IS_SUCCESSFUL           34299 non-null  int64 
dtypes: int64(4), object(8)
memory usage: 3.1+ MB


In [3]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(["EIN", "NAME", "SPECIAL_CONSIDERATIONS"], axis=1)
application_df.head()

KeyError: "['EIN', 'NAME', 'SPECIAL_CONSIDERATIONS'] not found in axis"

In [4]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE      17
AFFILIATION            6
CLASSIFICATION        71
USE_CASE               5
ORGANIZATION           4
STATUS                 2
INCOME_AMT             9
ASK_AMT             8747
IS_SUCCESSFUL          2
dtype: int64

In [5]:
application_df

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,142590,1
...,...,...,...,...,...,...,...,...,...
34294,T4,Independent,C1000,ProductDev,Association,1,0,5000,0
34295,T4,CompanySponsored,C3000,ProductDev,Association,1,0,5000,0
34296,T3,CompanySponsored,C2000,Preservation,Association,1,0,5000,0
34297,T5,Independent,C3000,ProductDev,Association,1,0,5000,1


In [6]:
# Look at APPLICATION_TYPE value counts for binning
application_counts = application_df['APPLICATION_TYPE'].value_counts()
application_counts

APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64

In [7]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = list(application_counts[application_counts < 500].index)

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: count, dtype: int64

In [8]:
# Look at CLASSIFICATION value counts for binning
classification_counts = application_df['CLASSIFICATION'].value_counts()
classification_counts

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: count, Length: 71, dtype: int64

In [58]:
# You may find it helpful to look at CLASSIFICATION value counts >1
classification_counts[classification_counts > 1]

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: count, dtype: int64

In [9]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = list(classification_counts[classification_counts < 1000].index)

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: count, dtype: int64

In [10]:
# Convert categorical data to numeric with `pd.get_dummies`
clean_df = pd.get_dummies(application_df)
clean_df

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M
0,1,5000,1,False,True,False,False,False,False,False,...,False,True,False,False,False,False,False,False,False,False
1,1,108590,1,False,False,False,True,False,False,False,...,False,False,True,False,False,False,False,False,False,False
2,1,5000,0,False,False,False,False,False,True,False,...,False,True,False,False,False,False,False,False,False,False
3,1,6692,1,False,False,False,True,False,False,False,...,True,False,False,True,False,False,False,False,False,False
4,1,142590,1,False,False,False,True,False,False,False,...,True,False,False,False,True,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,1,5000,0,False,False,False,False,True,False,False,...,False,True,False,False,False,False,False,False,False,False
34295,1,5000,0,False,False,False,False,True,False,False,...,False,True,False,False,False,False,False,False,False,False
34296,1,5000,0,False,False,False,True,False,False,False,...,False,True,False,False,False,False,False,False,False,False
34297,1,5000,1,False,False,False,False,False,True,False,...,False,True,False,False,False,False,False,False,False,False


In [11]:
# Split our preprocessed data into our features and target arrays
X = clean_df.drop('IS_SUCCESSFUL', axis=1)
y = clean_df['IS_SUCCESSFUL']

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [12]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [13]:
input_features = X_train_scaled.shape[1]
input_features

41

In [14]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 =  172
hidden_nodes_layer2 = 129
hidden_nodes_layers3 = 86
hidden_nodes_layers4 = 43

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))
# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))
# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layers3, activation="relu"))
# Fourth hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layers4, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 172)               7224      
                                                                 
 dense_1 (Dense)             (None, 129)               22317     
                                                                 
 dense_2 (Dense)             (None, 86)                11180     
                                                                 
 dense_3 (Dense)             (None, 43)                3741      
                                                                 
 dense_4 (Dense)             (None, 1)                 44        
                                                                 
Total params: 44506 (173.85 KB)
Trainable params: 44506 (173.85 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [15]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [16]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=200)

Epoch 1/200


804/804 [==============================] - 6s 5ms/step - loss: 0.5679 - accuracy: 0.7235
Epoch 2/200
804/804 [==============================] - 3s 4ms/step - loss: 0.5552 - accuracy: 0.7324
Epoch 3/200
804/804 [==============================] - 3s 4ms/step - loss: 0.5527 - accuracy: 0.7330
Epoch 4/200
804/804 [==============================] - 3s 4ms/step - loss: 0.5508 - accuracy: 0.7336
Epoch 5/200
804/804 [==============================] - 2s 3ms/step - loss: 0.5494 - accuracy: 0.7338
Epoch 6/200
804/804 [==============================] - 3s 4ms/step - loss: 0.5480 - accuracy: 0.7337
Epoch 7/200
804/804 [==============================] - 3s 4ms/step - loss: 0.5472 - accuracy: 0.7353
Epoch 8/200
804/804 [==============================] - 3s 4ms/step - loss: 0.5474 - accuracy: 0.7350
Epoch 9/200
804/804 [==============================] - 3s 3ms/step - loss: 0.5462 - accuracy: 0.7349
Epoch 10/200
804/804 [==============================] - 2s 3ms/step - loss: 0.5455 - accu

Epoch 77/200
804/804 [==============================] - 4s 5ms/step - loss: 0.5345 - accuracy: 0.7413
Epoch 78/200
804/804 [==============================] - 5s 6ms/step - loss: 0.5337 - accuracy: 0.7414
Epoch 79/200
804/804 [==============================] - 4s 5ms/step - loss: 0.5343 - accuracy: 0.7411
Epoch 80/200
804/804 [==============================] - 3s 4ms/step - loss: 0.5348 - accuracy: 0.7411
Epoch 81/200
804/804 [==============================] - 3s 4ms/step - loss: 0.5337 - accuracy: 0.7414
Epoch 82/200
804/804 [==============================] - 3s 4ms/step - loss: 0.5339 - accuracy: 0.7417
Epoch 83/200
804/804 [==============================] - 4s 5ms/step - loss: 0.5340 - accuracy: 0.7414
Epoch 84/200
804/804 [==============================] - 3s 4ms/step - loss: 0.5338 - accuracy: 0.7411
Epoch 85/200
804/804 [==============================] - 3s 4ms/step - loss: 0.5337 - accuracy: 0.7419
Epoch 86/200
804/804 [==============================] - 3s 4ms/step - loss: 0.5333

804/804 [==============================] - 3s 4ms/step - loss: 0.5314 - accuracy: 0.7412
Epoch 157/200
804/804 [==============================] - 5s 6ms/step - loss: 0.5317 - accuracy: 0.7418
Epoch 158/200
804/804 [==============================] - 4s 5ms/step - loss: 0.5315 - accuracy: 0.7415
Epoch 159/200
804/804 [==============================] - 3s 4ms/step - loss: 0.5598 - accuracy: 0.7410
Epoch 160/200
804/804 [==============================] - 3s 4ms/step - loss: 0.5325 - accuracy: 0.7410
Epoch 161/200
804/804 [==============================] - 3s 4ms/step - loss: 0.5315 - accuracy: 0.7418
Epoch 162/200
804/804 [==============================] - 3s 4ms/step - loss: 0.5313 - accuracy: 0.7421
Epoch 163/200
804/804 [==============================] - 3s 4ms/step - loss: 0.5312 - accuracy: 0.7421
Epoch 164/200
804/804 [==============================] - 3s 4ms/step - loss: 0.5317 - accuracy: 0.7420
Epoch 165/200
804/804 [==============================] - 3s 4ms/step - loss: 0.5311 - a

In [17]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5959 - accuracy: 0.7272 - 876ms/epoch - 3ms/step
Loss: 0.5959211587905884, Accuracy: 0.7272303104400635


In [68]:
# Export our model to HDF5 file
nn.save("Models/AlphabetSoupCharity-model3.h5")

C:\Users\Folio\anaconda3\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [69]:
nn.save("Models/AlphabetSoupCharity-model3.keras")